# Exercise 10 : Computing the static quark potential

### 

In [2]:
import numpy as np
import os
from functools import partial
from re import split
from time import time

def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in split(r'(\d+)', string_)]


def load_ensemble(action, beta, volume):
    #wilson_action or rectangular_action
    
    ensemble = []
    directory = "gauge_configs/" + str(action) + "/beta_" + str(beta) + "/vol_" + str(volume)  + "/"
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in sorted(files, key = natural_key):
            ensemble.append(np.load(os.path.abspath(os.path.join(root, name)), allow_pickle=True))
                        
    return ensemble

### Classes for the lattice and the site at each lattice point.

Site value is just a scalar here as this is a simple gauge simulation. The fundamental data structure of the lattice is a numpy array of site objects which have four links initialised to the identity matrix.

In [72]:
def link(lattice, coordinates, mu):
        
    n_points = lattice.shape[0]
        
    return lattice[coordinates[0] % n_points, coordinates[1] % n_points, coordinates[2] % n_points, coordinates[3] % n_points, mu, : , :]

### Rectangle loop lattice average

In [78]:
def wilson_loop(lattice, coordinates, spatial_extent, temporal_extent, spatial_direction):
    #each point has 3 temporal loops xt yt and zt, time is on exact same 
    #footing so doesnt really matter (euclidean isotropic)
    
    #starting bottom left, path of anti-quark in time ('static')
    bottom_res = link(lattice, coordinates, 0)
    bottom_coords_next = coordinates[:]
    
    #starting one in from top right as U^\dagger, path of quark in time ('static')
    top_coords = coordinates[:]
    top_coords[0] += temporal_extent-1
    top_coords[spatial_direction] += spatial_extent
    top_res = link(lattice, top_coords, 0).conj().T
    top_coords_next = top_coords[:]
    
    for i in range(t-1):
    #calculate total link product from top and bottom paths
        
        bottom_coords_next[0] += 1
        bottom_link_next = link(lattice, bottom_coords_next, 0)
        bottom_res = np.dot(bottom_res, bottom_link_next)
    
        top_coords_next[0] -= 1
        top_link_next = link(lattice, top_coords_next, 0).conj().T
        top_res = np.dot(top_res, top_link_next)
        

    #links for sides of path
    left_coords = coordinates[:]
    left_coords[spatial_direction] += spatial_extent-1
    left_res = link(lattice, left_coords, spatial_direction).conj().T
    left_coords_next = left_coords[:]
    
    right_coords = coordinates[:]
    right_coords[0] += temporal_extent
    right_res = link(lattice, right_coords, spatial_direction)
    right_coords_next = right_coords[:]
    
    for j in range(r-1):
        #calculate total link product from right and left paths
        
        left_coords_next[spatial_direction] -= 1
        left_link_next = link(lattice, left_coords_next, spatial_direction).conj().T
        left_res = np.dot(left_res, left_link_next)
        
        right_coords_next[spatial_direction] += 1
        right_link_next = link(lattice, right_coords_next, spatial_direction)
        right_res = np.dot(right_res, right_link_next)
        
    bottom_right = np.dot(bottom_res, right_res)
    top_left = np.dot(top_res, left_res)
    total_loop = np.dot(bottom_right, top_left)
    
    return np.trace(total_loop).real / 3

In [ ]:
def lattice_average_loop(lattice, spatial_extent, temporal_extent):
    dim = lattice.shape[0]
    res = 0
    for t in range(dim):
        for x in range(dim):
            for y in range(dim):
                for z in range(dim):
                    for sd in range(1,4):
                        res += wilson_loop(lattice, [t,x,y,z], spatial_extent, temporal_extent, sd)
                        
    return res / dim ** 4 / 3    

In [ ]:
def loop_array(lattice, spatial_extent):
    dim = lattice.shape[0]
    loop_arr = []
    for temporal_extent in range(dim):
        val = lattice_average_loop(lattice, spatial_extent, temporal_extent)
        loop_arr.append(val)
        
    return loop_arr        

In [79]:
def gauge_array(lattice_ensemble, spatial_exten):
    p = Pool(6)
    func = partial(loop_array, spatial_extent = spatial_extent)
    loop_ensemble = np.array(map(func, lattice_ensemble))
    p.terminate()
    
    return loop_ensemble

In [ ]:
def analysis():
    
    ensemble = load_ensemble(action, beta, volume)
    n_configs = ensemble.shape[x]
    dim = ensemble.shape[y]
    
    fig, axs =  plt.subplots(nrows=7, ncols=1, figsize=(8, 8), dpi=100)
    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    ax = axs.flat
    
    for spatial_extent in range(1, dim):
    
        ga = gauge_array(ensemble, spatial_extent)
        avg = np.mean(ga,0)
        err = np.std(ga,0) / np.sqrt(n_configs)
        avg_shift = np.roll(avg, -1)
        err_shift = np.roll(err, -1)
        
        V_eff = np.log(np.divide(avg,avg_shift))
        V_eff_err = np.sqrt(np.divide(err, avg)**2 + np.divide(err_shift, avg_shift)**2) / np.sqrt(n_configs)
    
        t = np.arange(0, dim)
        
        #Plotting
        ax[i].errorbar(t, V_eff, yerr=V_eff_err, fmt='o',  markersize=2, color='k')
        ax[i].tick_params(bottom="off", top="off", left="off", right="off")
        ax[i].set_title("Static Quark Potential at position: " + str(spatial_extent))
        #ax[i].set_xlim(0, 10)
        #ax[i].set_ylim(-2, 2)
        #ax[i].set_xticks([0, 2, 4, 6, 8, 10])
        #ax[i].set_yticks([-2, -1, 0, 1, 2])
        
    
    plt.show()

### Analysis